### **Avance 2 - Proyecto Final**

In [8]:
## Conectarse a la base de datos

from src.db.database import DBConnection
db1 = DBConnection()

In [2]:
# Ejecutar una query utilizando el método execute_query
# y guardar el resultado en un DataFrame de pandas

query = """
select FirstName as Nombre, LastName as Apellido, Gender as Genero, HireDate as FechaContratacion,
CityName as Ciudad, CountryName as Pais from employees e join cities c on e.cityid = c.cityid
join countries co on c.countryID= co.countryID LIMIT 5;
"""
df = db1.execute_query(query)
df

,Nombre,Apellido,Genero,FechaContratacion,Ciudad,Pais
0,Nicole,Fuller,F,2011-06-20 07:15:37,New Orleans,United States
1,Christine,Palmer,F,2011-04-27 04:07:57,Fremont,United States
2,Pablo,Cline,M,2012-03-30 18:55:23,Rochester,United States
3,Darnell,Nielsen,M,2014-03-06 06:55:03,Lubbock,United States
4,Desiree,Stuart,F,2014-11-16 22:59:55,Anaheim,United States


### **Patrones de diseño**

**Patron Singleton**

In [3]:
## Verificar Patron Singleton

db1 = DBConnection()
db2 = DBConnection()

print("Es la misma instancia?", db1 is db2)
print("Es la misma base de datos?", db1.engine.url == db2.engine.url)

Es la misma instancia? True
Es la misma base de datos? True


In [10]:
# Esto no creará una nueva conexión, sino que reutilizará la existente

db = DBConnection()

**Patron Factory**

In [11]:
## Implementación del patrón Factory

## SalesSummary
## Esta clase usa el patrón Factory y permite crear objetos de tipo SalesSummary a partir de un DataFrame de pandas. Lo que permite realizar una sola vez una query
## compleja y luego crear múltiples instancias de SalesSummary sin necesidad de repetir la consulta a la base de datos.

from src.design_patterns.factory import SalesSummary

query_sales = """
select SalesID, s.ProductID, ProductName, Quantity, TotalPrice, c.CustomerID, coalesce(concat( c.LastName, ", ", c.FirstName, " ", c.MiddleInitial, "."), "Sin nombre") as CustomerName,
e.EmployeeID, concat(e.LastName, ", ", e.FirstName, " ", e.MiddleInitial, ".") as EmployeeName
from sales s join products p on s.productid = p.ProductID
join customers c on s.CustomerID = c.CustomerID 
join employees e on s.SalesPersonID = e.EmployeeID
order by c.CustomerID;
"""
df_sales = db.execute_query(query_sales)
sales = [SalesSummary.from_series(row) for _, row in df_sales.iterrows()]
sales[0].__dict__

{'sale_id': 2494538,
 'product_id': 405,
 'product_name': 'Pastry - Raisin Muffin - Mini',
 'quantity': 1,
 'customer_id': 1,
 'customer_name': 'Frye, Stefanie Y.',
 'employee_id': 2,
 'employee_name': 'Palmer, Christine W.',
 'total_price': Decimal('2.00')}

In [7]:
## Implementación del patrón Factory

## CustomerLocationInfo
# Esta clase usa el patrón Factory y permite crear objetos de tipo CustomerLocationInfo a partir de un DataFrame de pandas. 
# Lo que permite realizar una sola vez una query

from src.design_patterns.factory import CustomerLocationInfo

## CustomerLocationInfo
query_customer_location = """
select CustomerID, FirstName, coalesce(MiddleInitial, "") as MiddleInitial, LastName, Address, CityName, CountryName
from customers cu join cities ci on cu.cityID = ci.CityID
join countries co on ci.CountryID = co.CountryID;
"""
df_customer_location = db.execute_query(query_customer_location)
customer_locations = [CustomerLocationInfo.from_series(row) for _, row in df_customer_location.iterrows()]
customer_locations[0].__dict__


{'customer_id': 103,
 'first_name': 'Jana',
 'middle_initial': 'Q',
 'last_name': 'Huang',
 'address': '841 Hague Road',
 'city_name': 'Dayton',
 'country_name': 'United States'}

In [15]:
## Implementación del patrón Strategy
# reutilizamos la query obtenida anteriormente para generar reportes de ventas totales por empleado.
# esta clase usa el patrón strategy y la estrategia es "TotalSalesByEmployee", que calcula las ventas totales por empleado.

from src.design_patterns.strategy import TotalSalesByEmployee

report = TotalSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,TotalVentas
17,18,"Bartlett, Warren C.",512154.00
20,21,"Brewer, Devon D.",618744.00
13,14,"Buckley, Wendi G.",406812.00
11,12,"Chen, Lindsay M.",338664.00
2,3,"Cline, Pablo Y.",83925.00
5,6,"Collins, Holly E.",171600.00
6,7,"Cook, Chadwick P.",195111.00
10,11,"Dickson, Sonya E.",308726.00
7,8,"Dyer, Julie E.",241680.00
14,15,"Finley, Kari D.",416940.00


In [16]:
## Implementación del patrón Strategy
# tambien reutilizamos la query obtenida anteriormente para generar reportes de ventas promedio por empleado.
# esta clase usa el patrón strategy y la estrategia es "AverageSalesByEmployee", que calcula las ventas promedio por empleado.

from src.design_patterns.strategy import AverageSalesByEmployee

report = AverageSalesByEmployee()
report.generate_report(df_sales,"EmployeeName")

,IDVendedor,Nombre Apellido Vendedor,Promedio de ventas
17,18,"Bartlett, Warren C.",232.269388
20,21,"Brewer, Devon D.",275.241993
13,14,"Buckley, Wendi G.",181.450491
11,12,"Chen, Lindsay M.",156.210332
2,3,"Cline, Pablo Y.",39.662098
5,6,"Collins, Holly E.",78.499543
6,7,"Cook, Chadwick P.",91.903439
10,11,"Dickson, Sonya E.",143.460037
7,8,"Dyer, Julie E.",106.326441
14,15,"Finley, Kari D.",197.414773


In [13]:
from src.design_patterns.strategy import ProductSalesByEmployee
report = ProductSalesByEmployee()
report.generate_report(df_sales, "ProductID", False)

,IDVendedor,Nombre Apellido Vendedor,Cantidad de productos vendidos
7,8,"Dyer, Julie E.",2273
1,2,"Palmer, Christine W.",2252
20,21,"Brewer, Devon D.",2248
13,14,"Buckley, Wendi G.",2242
15,16,"Walton, Chadwick U.",2205
17,18,"Bartlett, Warren C.",2205
22,23,"Flowers, Janet K.",2190
5,6,"Collins, Holly E.",2186
9,10,"Vang, Jean P.",2179
12,13,"Marks, Katina Y.",2177


In [25]:
## Implementación del patron Builder
# En esta celda se utiliza el patrón de diseño Builder para generar y combinar múltiples reportes de ventas a partir de un mismo DataFrame.
# ReportBuilder permite agregar diferentes estrategias de reporte (por ejemplo, ventas totales, promedio y por producto) 
# y construir todos los informes de manera flexible y escalable.
# Finalmente, se obtiene un reporte combinado ("CombinedReport") que integra los resultados de todas las estrategias aplicadas, 
# facilitando el análisis comparativo de la información.
# tambien puede generar reportes individuales como "TotalSalesByEmployee", "AverageSalesByEmployee" y "ProductSalesByEmployee".

from src.design_patterns.builder import ReportBuilder

builder = ReportBuilder()

reports = (
    builder.set_dataframe(df_sales)
    .set_combined_sorting("EmployeeName", True)
    .add_report(TotalSalesByEmployee())
    .add_report(AverageSalesByEmployee())
    .add_report(ProductSalesByEmployee())
    .build_all()
)

# Descomentar las siguientes líneas para ver los reportes individuales
# reports["ProductSalesByEmployee"]
# reports["TotalSalesByEmployee"]
# reports["AverageSalesByEmployee"]

# Reporte combinado que integra todos los reportes individuales
reports["CombinedReport"] 

,IDVendedor,Nombre Apellido Vendedor,TotalVentas,Promedio de ventas,Cantidad de productos vendidos
0,18,"Bartlett, Warren C.",512154.00,232.269388,2205
1,21,"Brewer, Devon D.",618744.00,275.241993,2248
2,14,"Buckley, Wendi G.",406812.00,181.450491,2242
3,12,"Chen, Lindsay M.",338664.00,156.210332,2168
4,3,"Cline, Pablo Y.",83925.00,39.662098,2116
5,6,"Collins, Holly E.",171600.00,78.499543,2186
6,7,"Cook, Chadwick P.",195111.00,91.903439,2123
7,11,"Dickson, Sonya E.",308726.00,143.460037,2152
8,8,"Dyer, Julie E.",241680.00,106.326441,2273
9,15,"Finley, Kari D.",416940.00,197.414773,2112


### **Tests**

In [ ]:
# Verificar que la clase DBConnection sigue el patrón Singleton
# Se ejecuta el test para verificar que la clase DBConnection sigue el patrón Singleton

!pytest tests/test_singleton_instance.py -v


============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 1 item

tests/test_singleton_instance.py::test_singleton_instance PASSED         [100%]

============================== 1 passed in 1.20s ==============================


In [ ]:
# Testando la clase SalesSummary que implementa el patrón Factory

!pytest tests/test_factory.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 3 items

tests/test_factory.py::test_sales_summary_creation PASSED                [ 33%]
tests/test_factory.py::test_customer_location_info_creation PASSED       [ 66%]
tests/test_factory.py::test_customer_location_info_missing_columns PASSED [100%]

============================== 3 passed in 0.44s ==============================


In [6]:
# Testeando las Clases que implementan el patrón Strategy:
# TotalSalesByEmployee, AverageSalesByEmployee y ProductSalesByEmployee

!pytest tests/test_strategy.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 5 items

tests/test_strategy.py::test_product_sales_by_employee PASSED            [ 20%]
tests/test_strategy.py::test_product_by_employee_sorting_by_name PASSED  [ 40%]
tests/test_strategy.py::test_product_by_employee_sorting_by_total_price PASSED [ 60%]
tests/test_strategy.py::test_total_sales_by_employee PASSED              [ 80%]
tests/test_strategy.py::test_average_sales_by_employee PASSED            [100%]

============================== 5 passed in 0.43s ==============================


In [26]:
# Teasteando la clase ReportBuilder que implementa el patrón Builder
# Esta clase permite generar reportes de ventas utilizando diferentes estrategias y combinarlos en un reporte final.

!pytest tests/test_builder.py -v

============================= test session starts =============================
platform win32 -- Python 3.13.3, pytest-8.3.5, pluggy-1.6.0 -- C:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final\env-final\Scripts\python.exe
cachedir: .pytest_cache
rootdir: c:\Users\veron\OneDrive\Documentos\curso Henry\Proyecto-Final
configfile: pytest.ini
collecting ... collected 1 item

tests/test_builder.py::test_report_builder_add_report PASSED             [100%]

============================== 1 passed in 0.41s ==============================


In [46]:
import importlib
import src.design_patterns.strategy
importlib.reload(src.design_patterns.strategy)

<module 'src.design_patterns.strategy' from 'c:\\Users\\veron\\OneDrive\\Documentos\\curso Henry\\Proyecto-Final\\src\\design_patterns\\strategy.py'>

In [64]:
import importlib
import src.design_patterns.builder
importlib.reload(src.design_patterns.builder)

<module 'src.design_patterns.builder' from 'c:\\Users\\veron\\OneDrive\\Documentos\\curso Henry\\Proyecto-Final\\src\\design_patterns\\builder.py'>